In [1]:
#Basic imports
import pandas as pd
import numpy as np
import pickle
from collections import defaultdict
import sys
from os import getcwd
import subprocess

curr_dir = getcwd()
#14.Final model utils functions
sys.path.append(curr_dir+"/../14.Final_Model/utils")
from final_model_funcs import ligands

curr_dir = getcwd()

In [2]:
predictions_path = curr_dir+"/2nd_level_pred/"

ens_dict = {"dna": "LIGAND", 
            "rna": "MODEL",
           "ion": "MODEL", 
           "peptide": "MODEL",
           "sm": "ALL"}

with open(curr_dir+"/../2.parse_Pfam/v32/updated_domain_to_pfam_acc_dict.pik", 'rb') as handle:
    domain_to_pfam_acc_dict = pickle.load(handle)

In [3]:
all_ligands_df = pd.DataFrame()
for ligand in ligands:
    
    #Reading all the ligand files
    cmd = "ls "+predictions_path+" | grep "+ligand+" | cut -d'_' -f3 | cut -d'.' -f1"
    ls_cmd_out = subprocess.Popen(cmd, shell=True, stdout=subprocess.PIPE) 
    
    ligand_df = pd.DataFrame()
    for line in ls_cmd_out.stdout.readlines():
        filename = line[:-1]
        ligand_curr_df = pd.read_csv(predictions_path+ligand+"_"+ens_dict[ligand]+"_"+filename+".csv", sep="\t",  index_col=0)
        
        if (ligand_df.shape[0]==0):
            ligand_df = ligand_curr_df
        else:
            ligand_df = pd.concat([ligand_df, ligand_curr_df])
    
    #Arranging the output table
    ligand_df["domain_name"], ligand_df["match_state"] = zip(*ligand_df["idx"].apply(lambda x: x.rsplit('_', 1)))
    del ligand_df["idx"]
    ligand_df["ligand"] = ligand

    #Get the columns to the original order
    cols = ligand_df.columns.tolist()
    cols = cols[0:1]+cols[3:4]+cols[2:3]+cols[1:2]
    ligand_df = ligand_df[cols]
    ligand_df.columns = ["binding_score", "ligand_type", "match_state", "domain_name"]
    
    #Add the Pfam id to the domain name
    pfam_acc_list = []
    for domain_name in ligand_df["domain_name"]:
        if (domain_name not in domain_to_pfam_acc_dict.keys()):
            print domain_name
        pfam_acc_list.append(domain_to_pfam_acc_dict[domain_name]+"_"+domain_name)
        
    ligand_df["pfam_id"] = pfam_acc_list
    del ligand_df["domain_name"]
    
    #Add to the combined dataframe of all the ligands
    if (all_ligands_df.shape[0]==0):
        all_ligands_df = ligand_df
    else:
        all_ligands_df = pd.concat([all_ligands_df, ligand_df])
    print "Finished "+ligand

Finished dna
Finished rna
Finished ion
Finished peptide
Finished sm


In [4]:
all_ligands_df["binding_score"] = all_ligands_df["binding_score"].round(5)

In [6]:
all_ligands_df[all_ligands_df["ligand_type"]=="sm"].sort_values(by="binding_score", ascending=False)

,binding_score,ligand_type,match_state,pfam_id
4481,0.98185,sm,57,PF00929_RNase_T
6659,0.97469,sm,183,PF00102_Y_phosphatase
4522,0.97058,sm,94,PF00929_RNase_T
6657,0.96892,sm,181,PF00102_Y_phosphatase
4476,0.96736,sm,52,PF00929_RNase_T
4484,0.96419,sm,6,PF00929_RNase_T
4480,0.96279,sm,56,PF00929_RNase_T
4462,0.95822,sm,4,PF00929_RNase_T
6654,0.94700,sm,179,PF00102_Y_phosphatase
314,0.94089,sm,207,PF00026_Asp


In [53]:
#Remove all scores below 0.001 to be 0
tmp_binding_scores = all_ligands_df["binding_score"]
adj_scores = [0 if x<0.001 else x for x in tmp_binding_scores]
all_ligands_df["binding_score"] = adj_scores

In [54]:
all_ligands_df.to_csv(curr_dir+"/Website_table/binding_scores_dsprint.csv", sep=",", index=False)

In [50]:
tmp_binding_scores.shape

(5567990,)

In [59]:
len(all_ligands_df["pfam_id"].unique())

6503